In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision import models
from torch.utils.data import DataLoader
import torch.nn.functional as F
import time
import datetime

from metrics.torch_seg_metrics import dice_score
from datasets.BRATS2018 import BRATS2018, NormalizeBRATS, ToTensor
from metrics.metrics import Evaluator
from models.unet import UNet

In [2]:
def time_stamp() -> str:
    ts = time.time()
    time_stamp = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    return time_stamp


In [3]:
def validate(model, seg_type='et', scan_type='t1ce'):
    val_transforms = transforms.Compose([
        NormalizeBRATS(),
        ToTensor()])

    BraTS_val_ds = BRATS2018('./BRATS2018',\
        data_set='val',\
        seg_type=seg_type,\
        scan_type=scan_type,\
        transform=val_transforms)

    data_loader = DataLoader(BraTS_val_ds, batch_size=2, shuffle=False, num_workers=0)

    running_dice_score = 0.

    for batch_ind, batch in enumerate(data_loader):
        imgs, targets = batch
        imgs = imgs.to(device)
        targets = targets.to(device)
        
        model.eval()

        with torch.no_grad():
            outputs = model(imgs)
            preds = torch.sigmoid(outputs)
            preds = preds > 0.5

            running_dice_score += dice_score(preds, targets) * targets.size(0)
            print('{} Batch {} running dice score: {:.6f}'.format(time_stamp(), batch_ind, running_dice_score))
    
    dice = running_dice_score / len(BraTS_val_ds)
    print('mean dice score of the validating set: {:.6f}'.format(dice))

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
wt_checkpoint = torch.load('../UNet_BraTS_DiceLoss_WT_manual/UNets-BRATS2018-DiceLoss-WT_batch6_lr1e-5_gamma1.0/terminated_model.tar',\
                           map_location=device)
wt_model = UNet(n_channels=1, n_classes=1)
wt_model.load_state_dict(wt_checkpoint['model_state_dict'])
wt_model.to(device)

UNet(
  (inp_conv): InConv(
    (conv): _DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace)
      )
    )
  )
  (down1): DownSamp(
    (down_samp): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): _DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(1

In [10]:
validate(wt_model, seg_type='wt', scan_type='flair')

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 0 running dice score: 2.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 1 running dice score: 4.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 2 running dice score: 6.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 3 running dice score: 8.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 4 running dice score: 10.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 5 running dice score: 12.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 6 running dice score: 14.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 7 running dice score: 16.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 8 running dice score: 18.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 9 running dice score: 20.000000
-Dice score- intersectio

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
Batch 80 running dice score: 133.079956
-Dice score- intersection: 0.00, preds: 0.00, targets: 13.00
Batch 81 running dice score: 133.080719
-Dice score- intersection: 0.00, preds: 0.00, targets: 35.00
Batch 82 running dice score: 133.081009
-Dice score- intersection: 0.00, preds: 0.00, targets: 62.00
Batch 83 running dice score: 133.081177
-Dice score- intersection: 0.00, preds: 0.00, targets: 117.00
Batch 84 running dice score: 133.081268
-Dice score- intersection: 52.00, preds: 89.00, targets: 236.00
Batch 85 running dice score: 133.721283
-Dice score- intersection: 333.00, preds: 445.00, targets: 509.00
Batch 86 running dice score: 135.117508
-Dice score- intersection: 833.00, preds: 1075.00, targets: 936.00
Batch 87 running dice score: 136.774399
-Dice score- intersection: 1261.00, preds: 1424.00, targets: 1362.00
Batch 88 running dice score: 138.584885
-Dice score- intersection: 1512.00, preds: 1665.00, targets: 1639.00


-Dice score- intersection: 3236.00, preds: 3412.00, targets: 3455.00
Batch 158 running dice score: 265.341187
-Dice score- intersection: 3574.00, preds: 3735.00, targets: 3778.00
Batch 159 running dice score: 267.244019
-Dice score- intersection: 3810.00, preds: 4009.00, targets: 3985.00
Batch 160 running dice score: 269.150452
-Dice score- intersection: 4011.00, preds: 4315.00, targets: 4161.00
Batch 161 running dice score: 271.043335
-Dice score- intersection: 4235.00, preds: 4505.00, targets: 4349.00
Batch 162 running dice score: 272.956604
-Dice score- intersection: 4215.00, preds: 4440.00, targets: 4312.00
Batch 163 running dice score: 274.883026
-Dice score- intersection: 4105.00, preds: 4311.00, targets: 4185.00
Batch 164 running dice score: 276.815704
-Dice score- intersection: 3978.00, preds: 4304.00, targets: 4063.00
Batch 165 running dice score: 278.717468
-Dice score- intersection: 3674.00, preds: 4312.00, targets: 3798.00
Batch 166 running dice score: 280.529541
-Dice scor

Batch 237 running dice score: 394.545654
-Dice score- intersection: 5753.00, preds: 5946.00, targets: 8362.00
Batch 238 running dice score: 396.153992
-Dice score- intersection: 5468.00, preds: 5719.00, targets: 8454.00
Batch 239 running dice score: 397.697205
-Dice score- intersection: 6176.00, preds: 6564.00, targets: 8693.00
Batch 240 running dice score: 399.316406
-Dice score- intersection: 6328.00, preds: 6550.00, targets: 8619.00
Batch 241 running dice score: 400.985077
-Dice score- intersection: 5811.00, preds: 5879.00, targets: 8090.00
Batch 242 running dice score: 402.649048
-Dice score- intersection: 5627.00, preds: 5711.00, targets: 7623.00
Batch 243 running dice score: 404.337067
-Dice score- intersection: 5533.00, preds: 5613.00, targets: 7328.00
Batch 244 running dice score: 406.047302
-Dice score- intersection: 5297.00, preds: 5380.00, targets: 7176.00
Batch 245 running dice score: 407.734772
-Dice score- intersection: 4979.00, preds: 5060.00, targets: 6881.00
Batch 246 

-Dice score- intersection: 2297.00, preds: 2398.00, targets: 2330.00
Batch 315 running dice score: 513.558105
-Dice score- intersection: 2399.00, preds: 2494.00, targets: 2442.00
Batch 316 running dice score: 515.502197
-Dice score- intersection: 2388.00, preds: 2542.00, targets: 2411.00
Batch 317 running dice score: 517.430725
-Dice score- intersection: 2343.00, preds: 2517.00, targets: 2354.00
Batch 318 running dice score: 519.354736
-Dice score- intersection: 2252.00, preds: 2486.00, targets: 2258.00
Batch 319 running dice score: 521.253540
-Dice score- intersection: 2148.00, preds: 2459.00, targets: 2161.00
Batch 320 running dice score: 523.113281
-Dice score- intersection: 1913.00, preds: 2285.00, targets: 1932.00
Batch 321 running dice score: 524.927856
-Dice score- intersection: 1451.00, preds: 2047.00, targets: 1490.00
Batch 322 running dice score: 526.568787
-Dice score- intersection: 767.00, preds: 1007.00, targets: 989.00
Batch 323 running dice score: 528.105835
-Dice score-

...

-Dice score- intersection: 4.00, preds: 4.00, targets: 984.00
Batch 8106 running dice score: 13324.293945
-Dice score- intersection: 633.00, preds: 667.00, targets: 1445.00
Batch 8107 running dice score: 13325.493164
-Dice score- intersection: 1589.00, preds: 1639.00, targets: 1859.00
Batch 8108 running dice score: 13327.310547
-Dice score- intersection: 1814.00, preds: 1820.00, targets: 2310.00
Batch 8109 running dice score: 13329.067383
-Dice score- intersection: 1999.00, preds: 1999.00, targets: 2628.00
Batch 8110 running dice score: 13330.795898
-Dice score- intersection: 2148.00, preds: 2148.00, targets: 2945.00
Batch 8111 running dice score: 13332.483398
-Dice score- intersection: 2219.00, preds: 2219.00, targets: 3289.00
Batch 8112 running dice score: 13334.094727
-Dice score- intersection: 2290.00, preds: 2290.00, targets: 3665.00
Batch 8113 running dice score: 13335.632812
-Dice score- intersection: 2436.00, preds: 2436.00, targets: 3861.00
Batch 8114 running dice score: 13337

-Dice score- intersection: 2156.00, preds: 2223.00, targets: 2483.00
Batch 8183 running dice score: 13462.600586
-Dice score- intersection: 2430.00, preds: 2552.00, targets: 2643.00
Batch 8184 running dice score: 13464.471680
-Dice score- intersection: 2551.00, preds: 2645.00, targets: 2819.00
Batch 8185 running dice score: 13466.338867
-Dice score- intersection: 2670.00, preds: 2739.00, targets: 2943.00
Batch 8186 running dice score: 13468.218750
-Dice score- intersection: 2708.00, preds: 2782.00, targets: 2968.00
Batch 8187 running dice score: 13470.102539
-Dice score- intersection: 2609.00, preds: 2678.00, targets: 2891.00
Batch 8188 running dice score: 13471.976562
-Dice score- intersection: 2564.00, preds: 2645.00, targets: 2808.00
Batch 8189 running dice score: 13473.857422
-Dice score- intersection: 2512.00, preds: 2637.00, targets: 2711.00
Batch 8190 running dice score: 13475.736328
-Dice score- intersection: 2295.00, preds: 2407.00, targets: 2491.00
Batch 8191 running dice sco

-Dice score- intersection: 3172.00, preds: 4063.00, targets: 3337.00
Batch 8259 running dice score: 13595.244141
-Dice score- intersection: 2367.00, preds: 3516.00, targets: 2550.00
Batch 8260 running dice score: 13596.804688
-Dice score- intersection: 1508.00, preds: 1982.00, targets: 1775.00
Batch 8261 running dice score: 13598.410156
-Dice score- intersection: 1026.00, preds: 1160.00, targets: 1317.00
Batch 8262 running dice score: 13600.067383
-Dice score- intersection: 872.00, preds: 1113.00, targets: 1041.00
Batch 8263 running dice score: 13601.686523
-Dice score- intersection: 619.00, preds: 900.00, targets: 745.00
Batch 8264 running dice score: 13603.191406
-Dice score- intersection: 49.00, preds: 79.00, targets: 542.00
Batch 8265 running dice score: 13603.506836
-Dice score- intersection: 0.00, preds: 0.00, targets: 415.00
Batch 8266 running dice score: 13603.506836
-Dice score- intersection: 0.00, preds: 0.00, targets: 14.00
Batch 8267 running dice score: 13603.507812
-Dice s

-Dice score- intersection: 5658.00, preds: 5741.00, targets: 5935.00
Batch 8336 running dice score: 13721.420898
-Dice score- intersection: 5382.00, preds: 5470.00, targets: 5694.00
Batch 8337 running dice score: 13723.349609
-Dice score- intersection: 4926.00, preds: 4973.00, targets: 5345.00
Batch 8338 running dice score: 13725.259766
-Dice score- intersection: 4791.00, preds: 4920.00, targets: 4977.00
Batch 8339 running dice score: 13727.196289
-Dice score- intersection: 4602.00, preds: 4793.00, targets: 4745.00
Batch 8340 running dice score: 13729.125977
-Dice score- intersection: 4260.00, preds: 4327.00, targets: 4481.00
Batch 8341 running dice score: 13731.060547
-Dice score- intersection: 3732.00, preds: 3776.00, targets: 4146.00
Batch 8342 running dice score: 13732.945312
-Dice score- intersection: 2920.00, preds: 2952.00, targets: 3520.00
Batch 8343 running dice score: 13734.750000
-Dice score- intersection: 2273.00, preds: 2286.00, targets: 2692.00
Batch 8344 running dice sco

-Dice score- intersection: 5037.00, preds: 5226.00, targets: 5245.00
Batch 8412 running dice score: 13858.598633
-Dice score- intersection: 4576.00, preds: 4744.00, targets: 4759.00
Batch 8413 running dice score: 13860.524414
-Dice score- intersection: 3992.00, preds: 4275.00, targets: 4112.00
Batch 8414 running dice score: 13862.428711
-Dice score- intersection: 3219.00, preds: 3744.00, targets: 3350.00
Batch 8415 running dice score: 13864.244141
-Dice score- intersection: 2380.00, preds: 2981.00, targets: 2536.00
Batch 8416 running dice score: 13865.969727
-Dice score- intersection: 1655.00, preds: 1841.00, targets: 1721.00
Batch 8417 running dice score: 13867.828125
-Dice score- intersection: 1097.00, preds: 1650.00, targets: 1097.00
Batch 8418 running dice score: 13869.425781
-Dice score- intersection: 520.00, preds: 1391.00, targets: 520.00
Batch 8419 running dice score: 13870.514648
-Dice score- intersection: 40.00, preds: 291.00, targets: 41.00
Batch 8420 running dice score: 138

In [5]:
tc_checkpoint = torch.load('../UNet_BraTS_DiceLoss_TC_manual/UNets-BRATS2018-TC-DiceLoss_batch6_training_epochs50_Adam_scheduler-step10-gamma1.0_lr1e-05_w_decay1e-05/terminated_model.tar',\
                           map_location=device)
tc_model = UNet(n_channels=1, n_classes=1)
tc_model.load_state_dict(tc_checkpoint['model_state_dict'])
tc_model.to(device)

UNet(
  (inp_conv): InConv(
    (conv): _DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace)
      )
    )
  )
  (down1): DownSamp(
    (down_samp): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): _DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(1

In [12]:
validate(tc_model, seg_type='tc', scan_type='t1ce')

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:37:59 Batch 0 running dice score: 2.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:37:59 Batch 1 running dice score: 4.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:00 Batch 2 running dice score: 6.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:00 Batch 3 running dice score: 8.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:00 Batch 4 running dice score: 10.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:00 Batch 5 running dice score: 12.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:00 Batch 6 running dice score: 14.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:01 Batch 7 running dice score: 16.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00


-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:13 Batch 69 running dice score: 122.397987
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:13 Batch 70 running dice score: 124.397987
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:14 Batch 71 running dice score: 126.397987
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:14 Batch 72 running dice score: 128.397980
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:14 Batch 73 running dice score: 130.397980
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:14 Batch 74 running dice score: 132.397980
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:14 Batch 75 running dice score: 134.397980
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:15 Batch 76 running dice score: 136.397980
-Dice score- intersection: 0.00, preds: 

2019-04-26 11:38:27 Batch 135 running dice score: 237.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:27 Batch 136 running dice score: 239.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:27 Batch 137 running dice score: 241.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:27 Batch 138 running dice score: 243.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:27 Batch 139 running dice score: 245.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:28 Batch 140 running dice score: 247.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:28 Batch 141 running dice score: 249.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:28 Batch 142 running dice score: 251.940857
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:28 Batch 143 ru

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:40 Batch 202 running dice score: 358.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:40 Batch 203 running dice score: 360.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:41 Batch 204 running dice score: 362.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:41 Batch 205 running dice score: 364.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:41 Batch 206 running dice score: 366.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:41 Batch 207 running dice score: 368.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:42 Batch 208 running dice score: 370.656677
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:42 Batch 209 running dice score: 372.656677
-Dice score- intersection: 0.00,

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:54 Batch 269 running dice score: 481.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:55 Batch 270 running dice score: 483.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:55 Batch 271 running dice score: 485.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:55 Batch 272 running dice score: 487.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:55 Batch 273 running dice score: 489.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:55 Batch 274 running dice score: 491.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:56 Batch 275 running dice score: 493.389618
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 11:38:56 Batch 276 running dice score: 495.389618
-Dice score- intersection: 0.00,

...

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:39 Batch 8166 running dice score: 13641.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:39 Batch 8167 running dice score: 13643.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:39 Batch 8168 running dice score: 13645.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:39 Batch 8169 running dice score: 13647.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:40 Batch 8170 running dice score: 13649.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:40 Batch 8171 running dice score: 13651.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:40 Batch 8172 running dice score: 13653.648438
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:40 Batch 8173 running dice score: 13655.648438
-Dice sc

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:52 Batch 8231 running dice score: 13752.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:53 Batch 8232 running dice score: 13754.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:53 Batch 8233 running dice score: 13756.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:53 Batch 8234 running dice score: 13758.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:53 Batch 8235 running dice score: 13760.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:54 Batch 8236 running dice score: 13762.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:54 Batch 8237 running dice score: 13764.649414
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:06:54 Batch 8238 running dice score: 13766.649414
-Dice sc

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:06 Batch 8297 running dice score: 13868.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:07 Batch 8298 running dice score: 13870.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:07 Batch 8299 running dice score: 13872.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:07 Batch 8300 running dice score: 13874.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:07 Batch 8301 running dice score: 13876.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:07 Batch 8302 running dice score: 13878.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:08 Batch 8303 running dice score: 13880.401367
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:08 Batch 8304 running dice score: 13882.401367
-Dice sc

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:20 Batch 8362 running dice score: 13979.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:20 Batch 8363 running dice score: 13981.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:20 Batch 8364 running dice score: 13983.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:21 Batch 8365 running dice score: 13985.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:21 Batch 8366 running dice score: 13987.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:21 Batch 8367 running dice score: 13989.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:21 Batch 8368 running dice score: 13991.839844
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:22 Batch 8369 running dice score: 13993.839844
-Dice sc

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:34 Batch 8427 running dice score: 14094.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:34 Batch 8428 running dice score: 14096.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:34 Batch 8429 running dice score: 14098.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:34 Batch 8430 running dice score: 14100.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:35 Batch 8431 running dice score: 14102.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:35 Batch 8432 running dice score: 14104.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:35 Batch 8433 running dice score: 14106.396484
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:07:35 Batch 8434 running dice score: 14108.396484
-Dice sc

In [5]:
et_checkpoint = torch.load('../UNet_BraTS_DiceLoss_ET_manual/UNets-BRATS2018-ET-DiceLoss_batch6_training_epochs50_Adam_scheduler-step10-gamma1.0_lr0.0001_w_decay1e-05/terminated_model.tar',\
                           map_location=device)
et_model = UNet(n_channels=1, n_classes=1)
et_model.load_state_dict(et_checkpoint['model_state_dict'])
et_model.to(device)

UNet(
  (inp_conv): InConv(
    (conv): _DoubleConv(
      (conv): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace)
      )
    )
  )
  (down1): DownSamp(
    (down_samp): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): _DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(1

In [6]:
validate(et_model, seg_type='et', scan_type='t1ce')

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:12 Batch 0 running dice score: 2.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:13 Batch 1 running dice score: 4.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:13 Batch 2 running dice score: 6.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:13 Batch 3 running dice score: 8.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:13 Batch 4 running dice score: 10.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:13 Batch 5 running dice score: 12.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:14 Batch 6 running dice score: 14.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:14 Batch 7 running dice score: 16.000000
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00


-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:26 Batch 69 running dice score: 118.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:26 Batch 70 running dice score: 120.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:27 Batch 71 running dice score: 122.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:27 Batch 72 running dice score: 124.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:27 Batch 73 running dice score: 126.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:27 Batch 74 running dice score: 128.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:27 Batch 75 running dice score: 130.408524
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:28 Batch 76 running dice score: 132.408524
-Dice score- intersection: 0.00, preds: 

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:40 Batch 136 running dice score: 225.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:40 Batch 137 running dice score: 227.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:40 Batch 138 running dice score: 229.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:40 Batch 139 running dice score: 231.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:41 Batch 140 running dice score: 233.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:41 Batch 141 running dice score: 235.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:41 Batch 142 running dice score: 237.876190
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:41 Batch 143 running dice score: 239.876190
-Dice score- intersection: 0.00,

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:53 Batch 203 running dice score: 344.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:54 Batch 204 running dice score: 346.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:54 Batch 205 running dice score: 348.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:54 Batch 206 running dice score: 350.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:54 Batch 207 running dice score: 352.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:55 Batch 208 running dice score: 354.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:55 Batch 209 running dice score: 356.954895
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:12:55 Batch 210 running dice score: 358.954895
-Dice score- intersection: 0.00,

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:08 Batch 270 running dice score: 462.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:08 Batch 271 running dice score: 464.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:08 Batch 272 running dice score: 466.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:08 Batch 273 running dice score: 468.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:09 Batch 274 running dice score: 470.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:09 Batch 275 running dice score: 472.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:09 Batch 276 running dice score: 474.093506
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:13:09 Batch 277 running dice score: 476.093506
-Dice score- intersection: 0.00,

...

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:40:56 Batch 8175 running dice score: 13650.487305
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:40:56 Batch 8176 running dice score: 13652.487305
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:40:57 Batch 8177 running dice score: 13654.487305
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:40:57 Batch 8178 running dice score: 13656.487305
-Dice score- intersection: 0.00, preds: 0.00, targets: 48.00
2019-04-26 12:40:57 Batch 8179 running dice score: 13656.487305
-Dice score- intersection: 178.00, preds: 243.00, targets: 228.00
2019-04-26 12:40:57 Batch 8180 running dice score: 13657.999023
-Dice score- intersection: 435.00, preds: 911.00, targets: 435.00
2019-04-26 12:40:57 Batch 8181 running dice score: 13659.291992
-Dice score- intersection: 562.00, preds: 1397.00, targets: 562.00
2019-04-26 12:40:58 Batch 8182 running dice score: 1

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:10 Batch 8240 running dice score: 13760.365234
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:10 Batch 8241 running dice score: 13762.365234
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:10 Batch 8242 running dice score: 13764.365234
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:11 Batch 8243 running dice score: 13766.365234
-Dice score- intersection: 0.00, preds: 0.00, targets: 48.00
2019-04-26 12:41:11 Batch 8244 running dice score: 13766.365234
-Dice score- intersection: 88.00, preds: 88.00, targets: 198.00
2019-04-26 12:41:11 Batch 8245 running dice score: 13767.595703
-Dice score- intersection: 264.00, preds: 272.00, targets: 316.00
2019-04-26 12:41:11 Batch 8246 running dice score: 13769.391602
-Dice score- intersection: 320.00, preds: 347.00, targets: 376.00
2019-04-26 12:41:11 Batch 8247 running dice score: 1377

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:24 Batch 8306 running dice score: 13880.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:24 Batch 8307 running dice score: 13882.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:24 Batch 8308 running dice score: 13884.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:25 Batch 8309 running dice score: 13886.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:25 Batch 8310 running dice score: 13888.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:25 Batch 8311 running dice score: 13890.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:25 Batch 8312 running dice score: 13892.426758
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:25 Batch 8313 running dice score: 13894.426758
-Dice sc

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:38 Batch 8371 running dice score: 13991.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:38 Batch 8372 running dice score: 13993.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:38 Batch 8373 running dice score: 13995.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:38 Batch 8374 running dice score: 13997.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:39 Batch 8375 running dice score: 13999.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:39 Batch 8376 running dice score: 14001.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:39 Batch 8377 running dice score: 14003.955078
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:39 Batch 8378 running dice score: 14005.955078
-Dice sc

-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:52 Batch 8436 running dice score: 14105.572266
-Dice score- intersection: 0.00, preds: 0.00, targets: 0.00
2019-04-26 12:41:52 Batch 8437 running dice score: 14106.572266
mean dice score of the validating set: 0.835945
